# PRÁCTICA 2

## Descripción

La base de datos, que se va a emplear en este trabajo, contiene mediciones hechas con un teléfono móvil inteligente. En concreto, el objetivo del problema es poder realizar el reconocimiento de la actividad que el sujeto está llevando a cabo. Esta base fue construida mediante las grabaciones recogidas de 30 sujetos con edades comprendidas entre los 19 y los 48 años en el marco de un estudio clínico. A los participantes se les solicito que realizaran actividades de la vida diaria mientras llevaban en la cintura mediante un cinturón un teléfono inteligente con sensores inerciales. El objetivo es clasificar las actividades en una de las seis actividades realizadas, es decir, WALKING, WALKINGUPSTAIRS, WALKINGDOWNSTAIRS, SITTING, STANDING, LAYING. Utilizando su acelerómetro y giroscopio integrados, se capturó la aceleración lineal en 3 ejes y la velocidad angular de esos mismos 3 ejes a una frecuencia constante de 50 Hz.

Las señales de los sensores (acelerómetro y giroscopio) se preprocesaron aplicando filtros de ruido y, a continuación, se muestrearon con la técnica de ventanas deslizantes de anchura fija de 2,56 segundos y con un solapamiento del 50% (128 lecturas/ventana). A partir de cada ventana, se obtuvo un vector de características calculando las variables tanto en el dominio del tiempo, como de la frecuencia. Cabe destacar que la señal de aceleración ha sido filtrada para eliminar la componente correspondiente a la gravedad y quedarse solo con el movimiento del cuerpo.


## Preparación de los datos:

### Carga de los datos junto con la descripción general del dominio:

* Cargar la base de datos proporcionada.
* Realizar una pequeña descripción exploratoria de los datos.

In [1]:
#! Primero de nada la instalación de los paquetes usados, en caso de ya tenerlos instalados
#! no ejecutar esta celda

using Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("Statistics")
# Pkg.add("CategoricalArrays")
# Pkg.add("MLJ")
# Pkg.add("MLJLinearModels")

In [2]:
# Cargar las bibliotecas necesarias
using DataFrames
using CSV
using Statistics

# 1. Cargar los datos
# Ruta del archivo CSV
ruta_archivo = "Datos/Datos_Práctica_Evaluación_1.csv"

# Leer los datos
datos = CSV.read(ruta_archivo, DataFrame)

# 2. Descripción general de los datos
println("Descripción General de los Datos\n")
println("Número de variables (columnas): ", ncol(datos))
println("Número de instancias (filas): ", nrow(datos))

# Primeras filas de los datos
println("\nPrimeras 5 filas del conjunto de datos:")
show(first(datos, 5), allcols=true)
println("\n\n")
# Calcular el número de individuos únicos
println("\nNúmero de individuos únicos: ", length(unique(datos.subject)))

# Clases de salida
clases = unique(datos.Activity)
println("\nNúmero de clases de salida: ", length(clases))
println("Clases de salida: ", clases)



Descripción General de los Datos

Número de variables (columnas): 563
Número de instancias (filas): 10299

Primeras 5 filas del conjunto de datos:
5×563 DataFrame
 Row │ subject  tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  tBodyAcc-max()-Y  tBodyAcc-max()-Z  tBodyAcc-min()-X  tBodyAcc-min()-Y  tBodyAcc-min()-Z  tBodyAcc-sma()  tBodyAcc-energy()-X  tBodyAcc-energy()-Y  tBodyAcc-energy()-Z  tBodyAcc-iqr()-X  tBodyAcc-iqr()-Y  tBodyAcc-iqr()-Z  tBodyAcc-entropy()-X  tBodyAcc-entropy()-Y  tBodyAcc-entropy()-Z  tBodyAcc-arCoeff()-X,1  tBodyAcc-arCoeff()-X,2  tBodyAcc-arCoeff()-X,3  tBodyAcc-arCoeff()-X,4  tBodyAcc-arCoeff()-Y,1  tBodyAcc-arCoeff()-Y,2  tBodyAcc-arCoeff()-Y,3  tBodyAcc-arCoeff()-Y,4  tBodyAcc-arCoeff()-Z,1  tBodyAcc-arCoeff()-Z,2  tBodyAcc-arCoeff()-Z,3  tBodyAcc-arCoeff()-Z,4  tBodyAcc-correlation()-X,Y  tBodyAcc-correlation()-X,Z  tBody

### Análisis de valores nulos
* Calcular el porcentaje de **valores nulos por variable**.
* Determinar el **porcentaje total** de **valores nulos** en el conjunto de datos.


In [3]:
# Análisis de valores nulos

# Función para calcular el porcentaje de valores nulos por columna
function porcentaje_nulos_por_columna(df::DataFrame)
    return [sum(ismissing.(df[!, col])) / nrow(df) * 100 for col in names(df)]
end

# 1. Calcular el porcentaje de valores nulos por variable
println("\nPorcentaje de valores nulos por variable:")
porcentajes_nulos = porcentaje_nulos_por_columna(datos)

# Mostrar los porcentajes en formato tabular
for (columna, porcentaje) in zip(names(datos), porcentajes_nulos)
    println("$(columna): $(round(porcentaje, digits=2))%")
end

# 2. Calcular el porcentaje total de valores nulos en todo el conjunto de datos
total_nulos = sum(sum(ismissing.(datos[!, col])) for col in names(datos))
porcentaje_total_nulos = total_nulos / (nrow(datos) * ncol(datos)) * 100

println("\nPorcentaje total de valores nulos en el conjunto de datos: ", round(porcentaje_total_nulos, digits=2), "%")



Porcentaje de valores nulos por variable:
subject: 0.0%
tBodyAcc-mean()-X: 0.0%
tBodyAcc-mean()-Y: 0.0%
tBodyAcc-mean()-Z: 0.0%
tBodyAcc-std()-X: 0.02%
tBodyAcc-std()-Y: 0.0%
tBodyAcc-std()-Z: 0.02%
tBodyAcc-mad()-X: 0.02%
tBodyAcc-mad()-Y: 0.03%
tBodyAcc-mad()-Z: 0.03%
tBodyAcc-max()-X: 0.02%
tBodyAcc-max()-Y: 0.0%
tBodyAcc-max()-Z: 0.01%
tBodyAcc-min()-X: 0.0%
tBodyAcc-min()-Y: 0.0%
tBodyAcc-min()-Z: 0.0%
tBodyAcc-sma(): 0.01%
tBodyAcc-energy()-X: 0.0%
tBodyAcc-energy()-Y: 0.0%
tBodyAcc-energy()-Z: 0.0%
tBodyAcc-iqr()-X: 0.03%
tBodyAcc-iqr()-Y: 0.01%
tBodyAcc-iqr()-Z: 0.0%
tBodyAcc-entropy()-X: 0.0%
tBodyAcc-entropy()-Y: 0.0%
tBodyAcc-entropy()-Z: 0.0%
tBodyAcc-arCoeff()-X,1: 0.0%
tBodyAcc-arCoeff()-X,2: 0.0%
tBodyAcc-arCoeff()-X,3: 0.0%
tBodyAcc-arCoeff()-X,4: 0.0%
tBodyAcc-arCoeff()-Y,1: 0.0%
tBodyAcc-arCoeff()-Y,2: 0.0%
tBodyAcc-arCoeff()-Y,3: 0.0%
tBodyAcc-arCoeff()-Y,4: 0.0%
tBodyAcc-arCoeff()-Z,1: 0.0%
tBodyAcc-arCoeff()-Z,2: 0.0%
tBodyAcc-arCoeff()-Z,3: 0.0%
tBodyAcc-arCoeff(

al haber columnas con porcentajes nulos, para hacer más simple el procesado de datos, haré un simple código para recoger cuales son las variables con valores núlos y clasificarlos en numéricos o categóricos, siendo así más sencillo de procesar después

In [4]:
# Detectar y mostrar columnas con valores nulos
println("Columnas con valores nulos:\n")

for columna in names(datos)
    if any(ismissing.(datos[!, columna]))  # Verifica si hay algún valor nulo en la columna
        println("Columna con valores nulos: ", columna)
    end
end



Columnas con valores nulos:

Columna con valores nulos: tBodyAcc-std()-X
Columna con valores nulos: tBodyAcc-std()-Z
Columna con valores nulos: tBodyAcc-mad()-X
Columna con valores nulos: tBodyAcc-mad()-Y
Columna con valores nulos: tBodyAcc-mad()-Z
Columna con valores nulos: tBodyAcc-max()-X
Columna con valores nulos: tBodyAcc-max()-Z
Columna con valores nulos: tBodyAcc-sma()
Columna con valores nulos: tBodyAcc-iqr()-X
Columna con valores nulos: tBodyAcc-iqr()-Y
Columna con valores nulos: tBodyAccJerk-std()-X
Columna con valores nulos: tBodyAccJerk-mad()-X
Columna con valores nulos: tBodyAccJerk-mad()-Y
Columna con valores nulos: tBodyAccJerk-mad()-Z
Columna con valores nulos: tBodyAccJerk-max()-X
Columna con valores nulos: tBodyAccJerk-sma()
Columna con valores nulos: tBodyAccJerk-energy()-X
Columna con valores nulos: tBodyAccJerk-energy()-Y
Columna con valores nulos: tBodyAccJerk-iqr()-Y
Columna con valores nulos: tBodyAccJerk-iqr()-Z
Columna con valores nulos: tBodyAccJerk-entropy()

In [5]:
# Detectar y mostrar columnas con valores nulos
println("Columnas con valores nulos y clasificables:\n")

for columna in names(datos)
    if any(ismissing.(datos[!, columna]))  # Verifica si hay algún valor nulo en la columna
        # Verifica si la columna tiene más de 50 valores únicos (son lo suficientemente grandes como para saber si no es categórico)
        if length(unique(datos[!, columna])) < 100
            clases = unique(datos[!, columna])
            println("Número de clases únicas en la columna ", columna, ": ", length(clases))
        end
    end
end


Columnas con valores nulos y clasificables:



Con esto, sabemos que unicamente hay valores nulos en los valores numéricos. por lo que ahora podemos tratarlos de una mejor forma.

In [6]:
columnas_valores_nulos = []  # Inicializa un arreglo vacío

for columna in names(datos)
    if any(ismissing.(datos[!, columna]))  # Verifica si hay algún valor nulo en la columna
        push!(columnas_valores_nulos, columna)  # Agrega la columna al arreglo
    end
end

println("Columnas con valores nulos: ", columnas_valores_nulos)


# Ahora tenemos un array con las columnas con valores nulos.


Columnas con valores nulos: Any["tBodyAcc-std()-X", "tBodyAcc-std()-Z", "tBodyAcc-mad()-X", "tBodyAcc-mad()-Y", "tBodyAcc-mad()-Z", "tBodyAcc-max()-X", "tBodyAcc-max()-Z", "tBodyAcc-sma()", "tBodyAcc-iqr()-X", "tBodyAcc-iqr()-Y", "tBodyAccJerk-std()-X", "tBodyAccJerk-mad()-X", "tBodyAccJerk-mad()-Y", "tBodyAccJerk-mad()-Z", "tBodyAccJerk-max()-X", "tBodyAccJerk-sma()", "tBodyAccJerk-energy()-X", "tBodyAccJerk-energy()-Y", "tBodyAccJerk-iqr()-Y", "tBodyAccJerk-iqr()-Z", "tBodyAccJerk-entropy()-X", "tBodyAccJerk-entropy()-Y", "tBodyAccJerk-entropy()-Z", "tBodyGyro-mad()-Y", "tBodyGyro-sma()", "tBodyGyro-iqr()-Y", "tBodyGyroJerk-std()-X", "tBodyGyroJerk-std()-Z", "tBodyGyroJerk-mad()-X", "tBodyGyroJerk-mad()-Z", "tBodyGyroJerk-sma()", "tBodyGyroJerk-iqr()-X", "tBodyGyroJerk-iqr()-Z", "tBodyGyroJerk-entropy()-X", "tBodyGyroJerk-entropy()-Y", "tBodyAccMag-std()", "tBodyAccMag-mad()", "tBodyAccMag-max()", "tBodyAccMag-sma()", "tBodyAccMag-energy()", "tBodyAccMag-iqr()", "tBodyAccMag-entropy(

### Preparar los datos para técnicas de clasificación
* Realizar las transformaciones necesarias en los datos.
* Rellenar los datos faltantes (si existen) utilizando una estrategia apropiada, como:
  * Media o mediana para variables numéricas.
  * Modo o una etiqueta específica para valores categóricos.


In [7]:
for columna in columnas_valores_nulos
    media_columna = mean(skipmissing(datos[!, columna]))  # Calcula la media ignorando nulos
    datos[!, columna] .= coalesce.(datos[!, columna], media_columna)  # Rellena nulos con la media
end

println("Valores nulos en columnas numéricas reemplazados por la media.")


Valores nulos en columnas numéricas reemplazados por la media.


In [8]:
# Detectar y mostrar columnas con valores nulos
println("Columnas con valores nulos:\n")

for columna in names(datos)
    if any(ismissing.(datos[!, columna]))  # Verifica si hay algún valor nulo en la columna
        println("Columna con valores nulos: ", columna)
    end
end


Columnas con valores nulos:



### Realizar un Holdout del 10% de los datos
* Reservar el 10% de los individuos (no instancias) como conjunto de prueba final.
* Utilizar la semilla 172 para asegurar la reproducibilidad.
* Mostrar cuáles son los individuos que quedan fuera del conjunto principal.

In [9]:
using Random  # Para controlar la semilla

# 1. Obtener los sujetos únicos
sujetos_unicos = unique(datos.subject)

# 2. Seleccionar aleatoriamente el 10% de los sujetos
semilla = 172
Random.seed!(semilla)  # Fijar la semilla para reproducibilidad

n_sujetos_holdout = Int(ceil(0.1 * length(sujetos_unicos)))  # Calcular el 10%
sujetos_holdout = shuffle(sujetos_unicos)[1:n_sujetos_holdout]  # Seleccionar aleatoriamente

println("Sujetos seleccionados para el conjunto de prueba (10%): ", sujetos_holdout)

# 3. Crear conjuntos de entrenamiento y prueba
datos_test = filter(row -> row.subject in sujetos_holdout, datos)  # Filtrar instancias de los sujetos seleccionados
datos_train = filter(row -> !(row.subject in sujetos_holdout), datos)  # El resto va al conjunto de entrenamiento

# 4. Mostrar resultados
println("\nNúmero de instancias en el conjunto de entrenamiento: ", nrow(datos_train))
println("Número de instancias en el conjunto de prueba: ", nrow(datos_test))


Sujetos seleccionados para el conjunto de prueba (10%): [4, 12, 29]

Número de instancias en el conjunto de entrenamiento: 9318
Número de instancias en el conjunto de prueba: 981


In [10]:
using Random

# 1. Obtener los sujetos únicos y dividirlos en 5 folds manualmente
sujetos_restantes = unique(datos_train.subject)  # Sujetos de los datos de entrenamiento
n_folds = 5

sujetos_mezclados = shuffle(sujetos_restantes)  # Mezclar sujetos

# Dividir los sujetos en 5 folds
folds = collect(Iterators.partition(sujetos_mezclados, ceil(Int, length(sujetos_mezclados) / n_folds)))


# 2. Función para normalizar los datos usando MinMaxScaler
function minmax_scaler(df::DataFrame)
    df_normalizado = copy(df)
    for col in names(df)
        if eltype(df[!, col]) <: Number  # Normalizar solo columnas numéricas
            min_val = minimum(skipmissing(df[!, col]))
            max_val = maximum(skipmissing(df[!, col]))
            df_normalizado[!, col] .= (df[!, col] .- min_val) ./ (max_val - min_val)
        end
    end
    return df_normalizado
end

# 3. Crear los 5 folds y aplicar MinMaxScaler
folds_data = []

for i in 1:n_folds
    # Sujetos en el fold actual
    sujetos_validacion = folds[i]
    
    # Crear conjuntos de validación y entrenamiento
    datos_validacion = filter(row -> row.subject in sujetos_validacion, datos_train)
    datos_entrenamiento = filter(row -> !(row.subject in sujetos_validacion), datos_train)
    
    # Normalizar ambos conjuntos
    datos_entrenamiento_norm = minmax_scaler(datos_entrenamiento)
    datos_validacion_norm = minmax_scaler(datos_validacion)
    
    # Almacenar los datos normalizados
    push!(folds_data, (train=datos_entrenamiento_norm, valid=datos_validacion_norm))
    
    println("Fold $i: Entrenamiento = ", nrow(datos_entrenamiento_norm), 
            " filas, Validación = ", nrow(datos_validacion_norm), " filas")
end

println("\nValidación cruzada 5-Fold completada y normalización aplicada.")



Fold 1: Entrenamiento = 7302 filas, Validación = 2016 filas
Fold 2: Entrenamiento = 7126 filas, Validación = 2192 filas
Fold 3: Entrenamiento = 7347 filas, Validación = 1971 filas
Fold 4: Entrenamiento = 7336 filas, Validación = 1982 filas
Fold 5: Entrenamiento = 8161 filas, Validación = 1157 filas

Validación cruzada 5-Fold completada y normalización aplicada.


# Segunda Parte de la Práctica:
## Creación de los Modelos Básicos:

### A continuación, realice las siguientes reducciones de dimensionalidad 
* NO aplicar ningún tipo de reducción 
* Filtrado ANOVA 
* Filtrado Mutual Information 
* Filtrado RFE con el método de LogisticRegression con una eliminación del  50% de las variables en cada pasada.

In [11]:
using CategoricalArrays

# Función para calcular ANOVA
function anova_f_score(X::DataFrame, y::CategoricalVector)
    scores = Float64[]  # Almacena los puntajes F
    y_vector = convert(Vector{String}, y)  # Convertir CategoricalVector a Vector{String}
    
    for columna in names(X)
        # Calcular varianza entre clases y dentro de clases
        clases = unique(y_vector)
        total_var = var(skipmissing(X[!, columna]))
        entre_clases_var = sum([
            (mean(X[y_vector .== clase, columna]) - mean(X[!, columna]))^2 * sum(y_vector .== clase)
            for clase in clases
        ]) / length(y_vector)
        
        dentro_clases_var = sum([
            var(skipmissing(X[y_vector .== clase, columna])) * sum(y_vector .== clase)
            for clase in clases
        ]) / length(y_vector)
        
        # Calcular el puntaje F
        F = entre_clases_var / dentro_clases_var
        push!(scores, F)
    end
    return scores
end

anova_f_score (generic function with 1 method)

In [12]:
using LinearAlgebra  # Para log2 seguro

# Función para calcular las probabilidades
function calcular_probabilidades(x::Vector, y::Vector)
    joint_counts = Dict()  # Frecuencia conjunta
    x_counts = Dict()      # Frecuencia marginal de x
    y_counts = Dict()      # Frecuencia marginal de y
    
    # Contar frecuencias conjuntas y marginales
    for (xi, yi) in zip(x, y)
        joint_counts[(xi, yi)] = get(joint_counts, (xi, yi), 0) + 1
        x_counts[xi] = get(x_counts, xi, 0) + 1
        y_counts[yi] = get(y_counts, yi, 0) + 1
    end
    
    n = length(x)  # Número total de observaciones
    joint_prob = Dict(k => v / n for (k, v) in joint_counts)
    x_prob = Dict(k => v / n for (k, v) in x_counts)
    y_prob = Dict(k => v / n for (k, v) in y_counts)
    
    return joint_prob, x_prob, y_prob
end

# Función para calcular Mutual Information
function mutual_information(x::Vector, y::Vector)
    joint_prob, x_prob, y_prob = calcular_probabilidades(x, y)
    mi = 0.0
    
    for ((xi, yi), p_xy) in joint_prob
        p_x = x_prob[xi]
        p_y = y_prob[yi]
        mi += p_xy * log2(p_xy / (p_x * p_y))
    end
    
    return mi
end

# Función para calcular MI para todas las características
function mutual_information_scores(X::DataFrame, y::CategoricalVector)
    scores = Float64[]
    y_vector = convert(Vector{String}, y)  # Convertir a Vector{String}
    
    for columna in names(X)
        feature_vector = X[!, columna]
        mi = mutual_information(feature_vector, y_vector)
        push!(scores, mi)
    end
    
    return scores
end

mutual_information_scores (generic function with 1 method)

In [16]:
using MLJBase
using MLJLinearModels
using DataFrames
using Statistics

function rfe(X::DataFrame, y::CategoricalVector; modelo=:logistic, k::Int=10, eliminar::Float64=0.5)
    remaining_features = names(X)  # Inicialmente todas las características
    y_vector = coerce(y, Multiclass)

    while length(remaining_features) > k
        println("Número de características restantes: ", length(remaining_features))

        # Seleccionar solo las características restantes
        X_subset = DataFrames.select(X, remaining_features)

        # Entrenar el modelo base
        if modelo == :logistic
            model = LogisticClassifier()
            mach = machine(model, X_subset, y_vector) |> fit!
        else
            error("Modelo no soportado")
        end

        # Obtener la importancia de las características
        params = fitted_params(mach)
        coefs = params.coefs  # Esto es una matriz (#clases - 1, #features)
        # Si es binario (2 clases), coefs es de tamaño (1, #features)
        # Si es multiclase, por ejemplo 6 clases, coefs es de tamaño (5, #features)

        if size(coefs, 1) == 1
            # Caso binario
            feature_coefs = coefs[1, :]
        else
            # Caso multiclase, por ejemplo promediamos la importancia
            feature_coefs = vec(mean(abs.(coefs), dims=1))
        end

        feature_importance = zip(remaining_features, abs.(feature_coefs))
        sorted_features = sort(collect(feature_importance), by=x -> x[2], rev=true)

        # Eliminar el 50% menos importante
        threshold = ceil(Int, length(remaining_features) * eliminar)
        remaining_features = [f[1] for f in sorted_features[1:end-threshold]]
    end

    println("Características seleccionadas por RFE: ", remaining_features)
    return DataFrames.select(X, remaining_features)
end



rfe (generic function with 1 method)

In [17]:
# Función preparar_datos
function preparar_datos(df::DataFrame; columna_objetivo::Symbol=:Activity, reduccion::String="none", k::Int=10)
    # Separar características y etiquetas
    X = DataFrames.select(df, Not(columna_objetivo))

    y = categorical(df[!, columna_objetivo])  # Convertir a CategoricalVector

    # Aplicar reducción de dimensionalidad (opcional)
    if reduccion == "none"
        println("No se aplicó reducción de dimensionalidad.")
    elseif reduccion == "ANOVA"
        println("Aplicando reducción de dimensionalidad con ANOVA.")
        scores = anova_f_score(X, y)
        top_k_indices = sortperm(scores, rev=true)[1:k]
        X = DataFrames.select(X, names(X)[top_k_indices])
        println("Características seleccionadas con ANOVA: ", names(X))
    elseif reduccion == "MI"
        println("Aplicando reducción de dimensionalidad con Mutual Information.")
        scores = mutual_information_scores(X, y)
        top_k_indices = sortperm(scores, rev=true)[1:k]
        X = DataFrames.select(X, names(X)[top_k_indices])
        println("Características seleccionadas con Mutual Information: ", names(X))
    elseif reduccion == "RFE"
        println("Aplicando reducción de dimensionalidad con RFE.")
        X = rfe(X, y; k=k)
    else
        println("Método de reducción no reconocido: ", reduccion)
    end

    return X, y
end


X, y = preparar_datos(datos_train, reduccion="none")
X_ANOVA, y = preparar_datos(datos_train, reduccion="ANOVA", k=15)
X_MI, y = preparar_datos(datos_train, reduccion="MI", k=10)
X_RFE, y = preparar_datos(datos_train, reduccion="RFE", k=10)


No se aplicó reducción de dimensionalidad.
Aplicando reducción de dimensionalidad con ANOVA.
Características seleccionadas con ANOVA: ["fBodyAccJerk-entropy()-X", "tGravityAcc-mean()-X", "tGravityAcc-min()-X", "tGravityAcc-max()-X", "tGravityAcc-energy()-X", "fBodyAccJerk-entropy()-Y", "tBodyAccJerkMag-entropy()", "fBodyBodyAccJerkMag-entropy()", "fBodyAcc-entropy()-X", "tBodyAccJerk-entropy()-X", "tBodyAcc-max()-X", "fBodyAccJerk-entropy()-Z", "tBodyAccJerk-entropy()-Z", "angle(X,gravityMean)", "tBodyAccJerk-entropy()-Y"]
Aplicando reducción de dimensionalidad con Mutual Information.
Características seleccionadas con Mutual Information: ["fBodyGyro-skewness()-Z", "tGravityAcc-arCoeff()-X,1", "tBodyGyroJerk-arCoeff()-X,3", "tBodyGyro-correlation()-X,Z", "tBodyAccJerk-arCoeff()-Z,4", "tBodyGyro-correlation()-Y,Z", "tBodyAccJerk-mean()-X", "fBodyAcc-skewness()-Z", "fBodyBodyGyroMag-skewness()", "tBodyAccMag-arCoeff()4"]
Aplicando reducción de dimensionalidad con RFE.
Número de caracterís

┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc MLJLinearModels.LogisticClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{Multiclass{6}}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}
└ @ MLJBase C:\Users\igran\.julia\packages\MLJBase\7nGJF\src\machines.jl:2

MethodError: MethodError: no method matching abs(::Pair{Symbol, SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}})
The function `abs` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  abs(!Matched::Bool)
   @ Base bool.jl:153
  abs(!Matched::Pkg.Resolve.FieldValue)
   @ Pkg C:\Users\igran\.julia\juliaup\julia-1.11.1+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Pkg\src\Resolve\fieldvalues.jl:51
  abs(!Matched::Missing)
   @ Base missing.jl:101
  ...
